In [3]:
import cv2
import numpy as np
import os
import csv
import pandas as pd
from batch_face import RetinaFace

In [2]:
!pip install batch_face


  Using cached batch_face-1.5.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached opencv_transforms-0.0.6-py3-none-any.whl.metadata (3.2 kB)
  Using cached sixdrepnet-0.1.6-py3-none-any.whl.metadata (10.0 kB)
Using cached batch_face-1.5.0-py3-none-any.whl (30.6 MB)
Using cached opencv_transforms-0.0.6-py3-none-any.whl (18 kB)
Using cached sixdrepnet-0.1.6-py3-none-any.whl (25 kB)


In [4]:
class VideoCamera(object):
    def __init__(self, path_video='', path_save_images='', label_df='', label_true='', conf=0.7):
        self.path_video = path_video
        self.path_save_images = path_save_images
        self.label_df = label_df
        self.label_true = label_true
        self.conf = conf # порог доставерности
        self.count_pie = 0 # подсчитываем количество кадров
        self.video = None
        self.h = None
        self.w = None

    def __del__(self):
        self.video.release()
        
    def get_h_w(self):
        try:
            (self.h, self.w) = self.fr.shape[:2]
        except AttributeError:
            print('The end')
            return False
        else: return True
            
    def get_frame(self):
        self.video = cv2.VideoCapture(self.path_video)
        total_frame = int(self.video.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = np.round(self.video.get(cv2.CAP_PROP_FPS))
        print('Number of frames: ', total_frame)
        print('FPS: ', fps)
        while True:
            _, self.fr = self.video.read()
            if self.get_h_w() is False: return None
            self.count_pie += 1
            name_img = str(self.count_pie).zfill(6)
            faces = detector(self.fr, cv=False)
            for f_id, box in enumerate(faces):
                box, _, prob = box
                if prob > self.conf:
                    startX = int(box[0])
                    startY = int(box[1])
                    endX = int(box[2])
                    endY = int(box[3])
                    (startX, startY) = (max(0, startX), max(0, startY))
                    (endX, endY) = (min(self.w - 1, endX), min(self.h - 1, endY))
                    cur_fr = self.fr[startY: endY, startX: endX]
                    
                    if not os.path.exists(self.path_save_images + str(f_id)):
                        os.makedirs(self.path_save_images + str(f_id))
                        df = pd.DataFrame(columns=['name_folder','path_images','emotion'])
                        df.to_csv(self.path_save_images + 'df_test.csv', index=False)
                            
                    cv2.imwrite(self.path_save_images + str(f_id) + "/{}.jpg".format(name_img), cur_fr)
                    
                    with open(self.path_save_images + 'df_test.csv', 'a+', newline='') as csvfile:
                        writer = csv.writer(csvfile, delimiter=',')
                        emo = self.label_true[self.label_df[os.path.basename(self.path_video).split('-')[2]]]
                        writer.writerow([os.path.basename(self.path_video).split('.')[0], self.path_save_images + str(f_id) + "/{}.jpg".format(name_img), emo])

In [5]:
import warnings
warnings.filterwarnings('ignore', category = UserWarning)
detector = RetinaFace(gpu_id=0)

In [6]:
%%time
label_ravdess = {'01':'Neutral', '02':'Calm', '03':'Happiness', '04':'Sadness', '05':'Anger', '06':'Fear', '07':'Disgust', '08':'Surprise'}
label_model = {'Neutral':0, 'Happiness':1, 'Sadness':2, 'Surprise':3, 'Fear':4, 'Disgust':5, 'Anger':6}

detect = VideoCamera(path_video='video/01-01-07-02-02-02-06.mp4', path_save_images='img/01-01-07-02-02-02-06/', label_df=label_ravdess, label_true=label_model, conf=0.7)
detect.get_frame()

Number of frames:  0
FPS:  0.0
The end
CPU times: total: 93.8 ms
Wall time: 237 ms
